In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


llm = ChatOpenAI(temperature=0.1)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir)

vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=cached_embeddings
)

retriever = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages([
  ("system", 
   """
    Use the following portion of a long document to see if any of the text is releevant to answer the questions. Return any relevant text verbatim.
    ---
    {context}
   """
   ),
  ("human","{question}")
])

map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
  documents = inputs["documents"]
  question = inputs["question"]

  return "\n\n".join(
    map_doc_chain.invoke({
      "question": question, 
      "context": document.page_content}).content
    for document in documents
  )
  
map_chain = {"documents": retriever, "question": RunnablePassthrough(),} | RunnableLambda(map_docs)


final_prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
     Given the following extracted parts of a long document and a question, create a final answer. If you don't know the answer, just say you don't know. Don't make up an answer.
     ---
     {context}
     """
     ),
    ("human", "{question}"),
])

chain = {"context": map_chain, "question": RunnablePassthrough()
         } | final_prompt | llm

chain.invoke("Where is Ishmael going to?")

AIMessage(content='Ishmael is going on a whaling voyage.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 132, 'total_tokens': 145, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5d6d766d-72a1-4730-ad46-051eb8f697fc-0')